In [1]:
import pyomo.environ as pyo
import random

In [2]:
path_solver = '../solvers/glpk-4.65/w64/glpsol.exe'
path_solver = '../solvers/cbc.exe'

In [3]:
demandas = [random.randint(10, 50) for d in range(13)]
capacidades = [random.randint(10, 50) for d in range(23)]
custo_fixo = [random.randint(400, 1000) for d in range(23)]
c = []
n_clientes = len(demandas)
n_fabricas = len(capacidades)

for i in range(len(capacidades)):
    cj = []
    for j in range(len(demandas)):
        cj.append(random.randint(100,300))    
    c.append(cj)
c


[[173, 201, 172, 116, 132, 250, 191, 230, 286, 286, 274, 135, 238],
 [161, 240, 299, 276, 219, 242, 177, 278, 196, 131, 211, 125, 223],
 [209, 279, 162, 254, 259, 156, 158, 284, 273, 250, 166, 221, 246],
 [197, 240, 162, 218, 150, 248, 114, 227, 241, 165, 164, 139, 292],
 [246, 128, 104, 225, 159, 296, 136, 125, 130, 143, 105, 208, 241],
 [257, 109, 259, 120, 101, 170, 165, 267, 171, 300, 184, 164, 281],
 [189, 238, 244, 163, 141, 110, 270, 218, 167, 202, 192, 142, 112],
 [244, 134, 275, 284, 139, 229, 264, 258, 156, 181, 137, 272, 157],
 [214, 299, 130, 110, 261, 152, 205, 164, 134, 210, 151, 287, 110],
 [249, 194, 206, 270, 141, 180, 116, 166, 218, 254, 160, 274, 154],
 [286, 153, 117, 261, 198, 166, 210, 149, 216, 107, 190, 292, 222],
 [200, 216, 240, 263, 104, 150, 238, 242, 220, 240, 159, 282, 212],
 [290, 182, 113, 227, 224, 181, 198, 228, 151, 155, 211, 245, 132],
 [276, 269, 129, 290, 271, 288, 284, 194, 189, 274, 266, 217, 245],
 [241, 251, 179, 227, 177, 258, 202, 296, 170, 1

In [4]:
capacidades[2], sum(demandas)

(14, 375)

In [5]:
c
demandas
capacidades

[41,
 33,
 14,
 20,
 23,
 47,
 36,
 38,
 26,
 17,
 33,
 18,
 13,
 49,
 28,
 37,
 21,
 24,
 14,
 25,
 36,
 26,
 43]

In [6]:
model = pyo.ConcreteModel()

In [7]:
model.I = pyo.RangeSet(n_fabricas)
model.J = pyo.RangeSet(n_clientes)

model.F = pyo.Param(model.I, initialize=lambda model, i: custo_fixo[i-1])
model.A = pyo.Param(model.J, initialize=lambda model, j: demandas[j-1])
model.B = pyo.Param(model.I, initialize=lambda model, i: capacidades[i-1])
model.C = pyo.Param(model.I, model.J, initialize=lambda model, i, j: c[i-1][j-1])


In [8]:
model.A.pprint()
sum(model.A.values())

model.B.pprint()
sum(model.A.values()),sum(model.B.values())

A : Size=13, Index=J, Domain=Any, Default=None, Mutable=False
    Key : Value
      1 :    33
      2 :    14
      3 :    44
      4 :    42
      5 :    10
      6 :    26
      7 :    18
      8 :    38
      9 :    17
     10 :    45
     11 :    37
     12 :    38
     13 :    13
B : Size=23, Index=I, Domain=Any, Default=None, Mutable=False
    Key : Value
      1 :    41
      2 :    33
      3 :    14
      4 :    20
      5 :    23
      6 :    47
      7 :    36
      8 :    38
      9 :    26
     10 :    17
     11 :    33
     12 :    18
     13 :    13
     14 :    49
     15 :    28
     16 :    37
     17 :    21
     18 :    24
     19 :    14
     20 :    25
     21 :    36
     22 :    26
     23 :    43


(375, 662)

In [9]:
model.X = pyo.Var(model.I, model.J, within=pyo.NonNegativeIntegers)
model.Y = pyo.Var(model.I, within=pyo.NonNegativeIntegers)

In [10]:
def Objt(model):
    return pyo.summation(model.F, model.Y) + pyo.summation(model.C,model.X) 
    #return sum(model.F[i]*model.Y[i] for i in model.I)# + pyo.summation(model.C,model.X) 
    #return sum(sum(model.C[i,j] for j in model.J)*model.Y[i] for i in model.I)

model.objetivo = pyo.Objective(rule=Objt, sense=pyo.minimize)

In [11]:
def rest_dem_cap(model,i):
    return sum(model.A[j]*model.X[i,j] for j in model.J) <= model.B[i]

def rest_cols(model,j):
    return sum(model.X[i,j] for i in model.I) == 1

def rest_cost_fix(model, i, j):
    return model.X[i,j]<=model.Y[i]

def rule_mochila(model):
    return sum(sum(model.C[i,j] for j in model.J)*model.Y[i] for i in model.I) <= 1500
#model.M = pyo.Constraint(rule=rule_mochila)

model.R1 = pyo.Constraint(model.I, rule=rest_dem_cap)
model.R2 = pyo.Constraint(model.J, rule=rest_cols)
model.R3 = pyo.Constraint(model.I, model.J, rule=rest_cost_fix)

In [12]:
#model.R1.pprint()
#model.R2.pprint()
#model.R3.pprint()
#model.M.pprint()
model.F.pprint()

F : Size=23, Index=I, Domain=Any, Default=None, Mutable=False
    Key : Value
      1 :   436
      2 :   458
      3 :   801
      4 :   903
      5 :   827
      6 :   472
      7 :   862
      8 :   531
      9 :   457
     10 :   961
     11 :   428
     12 :   458
     13 :   944
     14 :   855
     15 :   479
     16 :   680
     17 :   453
     18 :   473
     19 :   530
     20 :   680
     21 :   487
     22 :   929
     23 :   515


In [13]:
model.write('Facilities.lp', io_options={'symbolic_solver_labels':True})

('Facilities.lp', 2389264321456)

In [14]:
solver = pyo.SolverFactory('cbc', executable=path_solver)
sol = solver.solve(model, tee=False)

In [15]:
model.objetivo()

7812.0

In [19]:
for i in model.I:
    print(model.Y[i](), end=" ")

1.0 1.0 0.0 0.0 0.0 1.0 0.0 1.0 1.0 0.0 1.0 0.0 0.0 1.0 0.0 1.0 0.0 1.0 0.0 0.0 1.0 0.0 1.0 

In [16]:
for i in model.I:
    for j in model.J:
#        print(i,j,"-----",modelo.X[i,j](), end='')
        print(model.X[i,j](), end=' ')
    print("")

0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 
1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 
0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 
0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 
0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 
0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 
0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 
0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 
0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 
0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 
0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 
0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 
0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 
0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 
0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 
0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 
0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 
0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 
0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.

In [17]:
print(sol)


Problem: 
- Name: unknown
  Lower bound: 7812.0
  Upper bound: 7812.0
  Number of objectives: 1
  Number of constraints: 335
  Number of variables: 322
  Number of binary variables: 0
  Number of integer variables: 322
  Number of nonzeros: 322
  Sense: minimize
Solver: 
- Status: ok
  User time: -1.0
  System time: 0.9
  Wallclock time: 0.9
  Termination condition: optimal
  Termination message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 12
      Number of created subproblems: 12
    Black box: 
      Number of iterations: 3156
  Error rc: 0
  Time: 1.2636103630065918
Solution: 
- number of solutions: 0
  number of solutions displayed: 0

